In [5]:
from neo4j import GraphDatabase

In [7]:
session = GraphDatabase.driver("bolt://192.168.2.12:7687", auth=("neo4j", "admin"))

In [79]:
#cmd = "MATCH (cloudAtlas {title: 'Cloud Atlas'})<-[:DIRECTED]-(directors) RETURN directors.name"
#cmd = "MATCH (bacon:Person {name:'Kevin Bacon'})-[*1..4]-(hollywood) RETURN hollywood.title, hollywood.name"
cmd = "match (m:Movie)<-[:ACTED_IN]-(a:Person) return m.title as movie, collect(a.name) as cast limit 200"

nodes, rels = [], []
i = 0

def print_friends_of(tx, cmd):
    global i
    for record in tx.run(cmd):
        print(record)
        #print(record['hollywood.title'], record["hollywood.name"])
        
        nodes.append(
            {
                "title": record["movie"],
                "label": "movie"
            }
        )
        
        target = i
        i += 1
        
        for name in record['cast']:
            actor = {
                "title": name,
                "label": "actor"
            }
            
            try:
                source = nodes.index(actor)
            except Exception as e:
                nodes.append(actor)
                source = i
                i += 1
            
            rels.append(
                {
                    "source": source,
                    "target": target
                }
            )
    
    print({"nodes": nodes, "links": rels})

In [80]:
with session.session() as sess:
    sess.read_transaction(print_friends_of, cmd)

<Record movie='The Matrix' cast=['Emil Eifrem', 'Hugo Weaving', 'Laurence Fishburne', 'Carrie-Anne Moss', 'Keanu Reeves']>
<Record movie='The Matrix Reloaded' cast=['Hugo Weaving', 'Laurence Fishburne', 'Carrie-Anne Moss', 'Keanu Reeves']>
<Record movie='The Matrix Revolutions' cast=['Hugo Weaving', 'Laurence Fishburne', 'Carrie-Anne Moss', 'Keanu Reeves']>
<Record movie="The Devil's Advocate" cast=['Al Pacino', 'Charlize Theron', 'Keanu Reeves']>
<Record movie='A Few Good Men' cast=['James Marshall', 'Kevin Pollak', 'J.T. Walsh', 'Aaron Sorkin', 'Cuba Gooding Jr.', 'Christopher Guest', 'Noah Wyle', 'Kiefer Sutherland', 'Kevin Bacon', 'Demi Moore', 'Jack Nicholson', 'Tom Cruise']>
<Record movie='Top Gun' cast=['Val Kilmer', 'Meg Ryan', 'Tom Skerritt', 'Kelly McGillis', 'Tom Cruise', 'Anthony Edwards']>
<Record movie='Jerry Maguire' cast=["Jerry O'Connell", 'Bonnie Hunt', 'Jay Mohr', 'Cuba Gooding Jr.', 'Jonathan Lipnicki', 'Renee Zellweger', 'Kelly Preston', 'Regina King', 'Tom Cruise'

In [ ]:
with session.session() as sess:
    sess.read_transaction(print_friends_of, "Alice")